**Beta distribution**

Example problem to examine proportions.

A frequentist approach https://vasishth.github.io/Freq_CogSci/from-the-paired-t-test-to-the-linear-mixed-model.html

**I'll stop this for now. I think it may not be high ROI for me to keep working on this. I'll keep doing SR problems.**

In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import seaborn as sns
import daft
from causalgraphicalmodels import CausalGraphicalModel

from scipy.optimize import curve_fit

In [2]:
%load_ext nb_black
%config InlineBackend.figure_format = 'retina'
%load_ext watermark
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")
az.rcParams["stats.hdi_prob"] = 0.89  # sets default credible interval used by arviz

<IPython.core.display.Javascript object>

In [3]:
sns.set_context("talk")

<IPython.core.display.Javascript object>

In [4]:
def standardize(x):
    x = (x - np.mean(x)) / np.std(x)
    return x

<IPython.core.display.Javascript object>

# A simple dataset taken from Wikipedia

The data I'll use is taken from [Wikipedia's entry on the paired t-test](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples). It's a simple set with four subjects taking a test twice, an example of repeated measures.  Interestingly, in the Chapter 1 introduction and in Chapter 13, Dr. McElreath highlights multi-level models as a way to get "improved estimates for repeat sampling".

In [5]:
df_tests = pd.DataFrame(
    {
        "Name": ["Mike", "Melanie", "Melissa", "Mitchell"],
        "Test_1": [35, 50, 90, 78],
        "Test_2": [67, 46, 86, 91],
    }
)

<IPython.core.display.Javascript object>

In [6]:
df_tests

,Name,Test_1,Test_2
0,Mike,35,67
1,Melanie,50,46
2,Melissa,90,86
3,Mitchell,78,91


<IPython.core.display.Javascript object>

# Calculate the t-statistic

The paired t-test is calculating whether the average of the difference is significantly different. We can get this quickly with a Python function.

## T-statistic using scipy.stats

In [7]:
stats.ttest_rel(df_tests["Test_1"], df_tests["Test_2"])

Ttest_relResult(statistic=-1.0784834690588145, pvalue=0.3598054860063756)

<IPython.core.display.Javascript object>

## T-statistic calculated manually

The formula for the t-statistic is given on the same Wikipedia page.

$$t = \frac{\bar{X}_D - \mu_0}{s_D / \sqrt{n}}$$

where $\bar{X}_D$ is the average of the differences between all pairs and $s_D$ is the standard deviation of the differences. We can set $mu_0$ to 0 if we want to test whether the average is significantly different. This is straightforward to calculate.

In [8]:
diff = df_tests["Test_1"] - df_tests["Test_2"]

<IPython.core.display.Javascript object>

In [9]:
t_stat = diff.mean() / (diff.std() / np.sqrt(len(df_tests)))
print("Manual calculated t-statistic: ", t_stat)

Manual calculated t-statistic:  -1.0784834690588145


<IPython.core.display.Javascript object>

We get exactly the same answer as `scipy.stats` so that is comforting.

# Calculate the t-statistic with a Bayesian approach

Now the fun part. We can set this up as a Bayesian multi-level linear model. A non-Bayesian approach for connecting the paired t-test to a linear mixed model was described [here](https://vasishth.github.io/Freq_CogSci/from-the-paired-t-test-to-the-linear-mixed-model.html).

First, let's do some table reformatting, so that we can use it in our model. We'll represent name and times into numerical codes and standardize the test scores.

In [11]:
df_tests2 = pd.melt(df_tests, id_vars="Name")
df_tests2.columns = ["Name", "Time", "Score"]
df_tests2["Name_code"] = pd.Categorical(df_tests2["Name"]).codes
df_tests2["Time_code"] = pd.Categorical(df_tests2["Time"]).codes
df_tests2["Score_std"] = standardize(df_tests2["Score"])
df_tests2

,Name,Time,Score,Name_code,Time_code,Score_std
0,Mike,Test_1,35,2,0,-1.610167
1,Melanie,Test_1,50,0,0,-0.875490
2,Melissa,Test_1,90,1,0,1.083649
3,Mitchell,Test_1,78,3,0,0.495907
4,Mike,Test_2,67,2,1,-0.042856
5,Melanie,Test_2,46,0,1,-1.071404
6,Melissa,Test_2,86,1,1,0.887735
7,Mitchell,Test_2,91,3,1,1.132627


<IPython.core.display.Javascript object>

There are different "clusters" here. On one level, is time, whether it was the first or second test. On another level are the individual subjects.

## Non multi-level model with time as an indexed, categorical variable

Let's start without a multi-level model. Since the main question is whether there are differences between the first and second tests, we'll use only time as a cluster for now and ignore the subject cluster. This is an intercept only model, with time represented as an indexed categorical variable.

<span style="color:red">Not sure if I set this up right. I think we'd still use a Normal here but should I use a Student t distribution instead. Also I used a flat prior since non-Bayesians wouldn't regularize </span>.

$$\text{score}_i \text{ ~ Normal}(\mu_i, \sigma) $$
$$\mu_i = \alpha_{\text{TIME}}$$
$$\alpha_j \text{ ∼ Normal}(0, 10)$$
$$\sigma \text{ ~ Exp}(1)$$

In [12]:
with pm.Model() as m1:

    # prior for SD of testers
    sigma = pm.Exponential("sigma", 1.0)

    # regularizing prior
    a = pm.Normal("a", 0, 10, shape=2)  # two time points

    # mu is deterministic, equivalent to alpha indexed by time
    mu = a[df_tests2["Time_code"]]

    # likelihood
    score = pm.Normal("score", mu=mu, sd=sigma, observed=df_tests2["Score_std"])

    trace_m1 = pm.sample(
        draws=1000, random_seed=19, return_inferencedata=True, progressbar=False
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, sigma]
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 20 seconds.


<IPython.core.display.Javascript object>

In [13]:
az.summary(trace_m1)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a[0],-0.235,0.638,-1.242,0.741,0.013,0.011,2353.0,1753.0,2439.0,2051.0,1.0
a[1],0.239,0.644,-0.723,1.308,0.012,0.011,2907.0,1690.0,2995.0,2179.0,1.0
sigma,1.238,0.381,0.703,1.730,0.009,0.006,1965.0,1917.0,2084.0,1943.0,1.0


<IPython.core.display.Javascript object>

### McElreath's way of calculating contrast

On page 158 of McElreath's text, he describes how to calculate the **contrast**. It references the Pythagorean theorem of statistics. In this problem, we would do the following.

In [17]:
a0_mean = az.summary(trace_m1).loc["a[0]", "mean"]
a1_mean = az.summary(trace_m1).loc["a[1]", "mean"]
a0_sd = az.summary(trace_m1).loc["a[0]", "sd"]
a1_sd = az.summary(trace_m1).loc["a[1]", "sd"]

<IPython.core.display.Javascript object>

In [20]:
print(
    "{0:0.3f} +/- {1:0.3f}".format(a1_mean - a0_mean, np.sqrt(a1_sd ** 2 + a0_sd ** 2))
)

0.474 +/- 0.907


<IPython.core.display.Javascript object>

Then the t-statistic calculation would be:

In [21]:
0.474 / (0.907 / np.sqrt(4))

1.0452039691289967

<IPython.core.display.Javascript object>

### old way, possibly incorrect

Now let's calculate the contrast between the alpha parameters representing the two timepoints (??)

In [14]:
# Put the trace object in a dataframe so we can pull out the posteriors
trace_m1_df = trace_m1.to_dataframe()

# calculate contrast
post_diff = (
    trace_m1_df[("posterior", "a[0]", 0)] - trace_m1_df[("posterior", "a[1]", 1)]
)

<IPython.core.display.Javascript object>

<span style="color:red">With a pretty flat prior (what non-Bayesians would do), I got pretty close to the answer, but not sure if I just lucky</span>.

## Multi-level model

We can now recognize both clusters and let the model do adaptive regularization.

$$\text{score}_i \text{ ~ Normal}(\mu_i, \sigma_1) $$
$$\mu_i = \alpha_{\text{TIME}} + \gamma_{\text{SUBJECT}} $$
$$\alpha_j \text{ ∼ Normal}(\bar{\alpha}, \sigma_\alpha)$$
$$\gamma_j \text{ ∼ Normal}(0, \sigma_\gamma) $$
$$\bar{\alpha} \text{ ∼ Normal}(0, 10)$$
$$\sigma_1 \text{ ~ Exp}(1)$$
$$\sigma_\alpha \text{ ~ Exp}(1)$$
$$\sigma_\gamma \text{ ~ Exp}(1)$$


Alpha is using an adaptive prior for time (j=1,2) and gamma is using adaptive prior for subject (j = 1..4). We'll use one global mean parameter ("bar_alpha") since each varying intercept type will be added to the same linear prediction. This global mean parameter is essentially the average across all tests and subjects. Like in the previous model, it is pretty wide to represent the non-Bayesian perspective. Each cluster also gets its own sigma.

In [22]:
with pm.Model() as m2:
    # Top line sigma
    sigma1 = pm.Exponential("sigma1", 1.0)

    # prior for average person and timepoint
    a_bar = pm.Normal("a_bar", 0.0, 10)

    # prior for SD of testers and time
    sigma_a = pm.Exponential("sigma_a", 1.0)
    sigma_g = pm.Exponential("sigma_g", 1.0)

    # adaptive priors
    a = pm.Normal("a", a_bar, sigma_a, shape=2)  # 2 time points
    g = pm.Normal("g", 0, sigma_g, shape=4)  # 4 subjects

    mu = a[df_tests2["Time_code"]] + g[df_tests2["Name_code"]]

    # likelihood
    score = pm.Normal("score", mu=mu, sd=sigma1, observed=df_tests2["Score_std"])

    trace_m2 = pm.sample(
        draws=1000, random_seed=19, return_inferencedata=True, progressbar=False
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [g, a, sigma_g, sigma_a, a_bar, sigma1]
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 20 seconds.
There were 88 divergences after tuning. Increase `target_accept` or reparameterize.
There were 118 divergences after tuning. Increase `target_accept` or reparameterize.
There were 96 divergences after tuning. Increase `target_accept` or reparameterize.
There were 125 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7129996194220574, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 10% for some parameters.


<IPython.core.display.Javascript object>

<span style="color:red">The divergences indicate I need to re-paramaterize, but let me know if my overall approach is right</span>

In [54]:
az.summary(trace_m2)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a_bar,-0.038,0.926,-1.355,1.272,0.040,0.028,549.0,549.0,529.0,924.0,1.01
a[0],-0.162,0.630,-1.103,0.822,0.026,0.019,575.0,575.0,565.0,820.0,1.01
a[1],0.126,0.634,-0.873,1.082,0.027,0.019,544.0,544.0,532.0,846.0,1.00
g[0],-0.617,0.682,-1.603,0.413,0.028,0.020,589.0,589.0,601.0,836.0,1.00
g[1],0.675,0.683,-0.322,1.739,0.029,0.021,548.0,528.0,556.0,888.0,1.01
g[2],-0.531,0.670,-1.614,0.395,0.027,0.020,612.0,568.0,647.0,751.0,1.00
g[3],0.535,0.659,-0.388,1.611,0.029,0.021,529.0,486.0,533.0,506.0,1.01
sigma,0.784,0.338,0.324,1.210,0.013,0.009,682.0,682.0,620.0,809.0,1.00
sigma_a,0.749,0.692,0.107,1.527,0.030,0.021,527.0,527.0,246.0,169.0,1.01
sigma_g,0.943,0.548,0.127,1.631,0.020,0.014,751.0,751.0,574.0,598.0,1.01


<IPython.core.display.Javascript object>

### McElreath's way of calculating contrast

In [23]:
a0_mean = az.summary(trace_m2).loc["a[0]", "mean"]
a1_mean = az.summary(trace_m2).loc["a[1]", "mean"]
a0_sd = az.summary(trace_m2).loc["a[0]", "sd"]
a1_sd = az.summary(trace_m2).loc["a[1]", "sd"]

<IPython.core.display.Javascript object>

In [24]:
print(
    "{0:0.3f} +/- {1:0.3f}".format(a1_mean - a0_mean, np.sqrt(a1_sd ** 2 + a0_sd ** 2))
)

0.288 +/- 0.894


<IPython.core.display.Javascript object>

Then the t-statistic calculation would be:

In [25]:
0.288 / (0.894 / np.sqrt(4))

0.6442953020134228

<IPython.core.display.Javascript object>

### old way of getting t-statistic, possibly incorrect

In [56]:
# Put the trace object in a dataframe so we can pull out the posteriors
trace_m2_df = trace_m2.to_dataframe()

# calculate contrast
post_diff2 = (
    trace_m2_df[("posterior", "a[0]", 0)] - trace_m2_df[("posterior", "a[1]", 1)]
)

<IPython.core.display.Javascript object>

<span style="color:red">Again, I'm just using the t-statistic equation here</span>.

In [61]:
post_diff2.mean() / (post_diff2.std() / np.sqrt(4))

-1.2088091266973582

<IPython.core.display.Javascript object>

<span style="color:red">I get an answer further than the previous model but in the right direction. Again, not sure if I had done something incorrect</span>

Appendix: Environment and system parameters

In [123]:
%watermark -n -u -v -iv -w

Last updated: Wed Aug 04 2021

Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.20.0

seaborn   : 0.11.1
scipy     : 1.6.0
matplotlib: 3.3.4
arviz     : 0.11.1
daft      : 0.1.0
pandas    : 1.2.1
pymc3     : 3.11.0
json      : 2.0.9
numpy     : 1.20.1

Watermark: 2.1.0



<IPython.core.display.Javascript object>